In [75]:
#import ee
#ee.Authenticate()

In [18]:
import ee
import geemap
# metodo de clasificacion
import jenkspy

import geopandas as gpd
from geopandas.tools import overlay
from shapely.geometry import Polygon

import pandas as pd
import numpy as np

import openpyxl

In [26]:
ee.Initialize()

In [27]:
ruta_catastro='projects/ee-bismarksr17/assets/CATASTRO_2023-02-25'
ruta_ndvi = 'projects/ee-bismarksr17/assets/NDVI_01_2023'

CODIGO_PROPIEDAD='unidad_01'
NOM_PROPIEDAD='unidad_02'
CODIGO_CANHERO='unidad_03'
NOM_CANHERO='unidad_04'

VARIEDAD='variedad'
ESTADO='RENOVACION'
SUPERFICIE='area'

In [28]:
def asig_cat(label):
    if label == 1:
        return 0
    else:
        return label * 10

vis_params_caña = {
    'color': 'red', 
    'width': 2,
    'lineType': 'solid',
    'fillColor': '00000000',
}

# parametro de visualizacion LOTES RENOVACION
vis_params_renov = {
    'color': 'blue', 
    'width': 2,
    'lineType': 'solid',
    'fillColor': '00000000',
}

In [29]:
#fase 1
#lista_props = [300, 1729, 1454, 30, 325, 17, 629, 19, 275, 256, 1728, 114, 949, 257, 29, 551, 31, 221, 92, 47, 478, 1618, 10, 1727, 700, 123, 259, 297, 591, 1314, 365, 239, 1450, 201, 674, 122, 216, 205, 495, 217, 375, 5, 33, 895, 63, 658, 23, 50, 1528, 1831, 185, 606, 343, 326, 510, 74, 132, 1011, 333, 327, 242, 299, 98, 1704, 294, 113, 560, 1215, 55, 306, 277, 752, 197, 209]
#face 2
#lista_props = [111, 320, 1486, 1173, 13, 18, 1106, 555, 219, 1421, 429, 222, 126, 184, 308, 416, 218, 1149, 270, 1705, 913, 253, 284, 415, 22, 781, 54, 118, 1743, 27, 100, 1466, 1487, 519, 483, 1801, 1625, 668, 1482, 1624, 504, 1015, 328, 771, 1289, 669, 1481, 1707, 437, 258, 729, 921, 791, 161, 1852, 1591, 1007, 301, 273, 172, 274, 590, 1480, 1222, 1467, 85, 1, 727, 1768, 226, 1604, 198, 251, 238, 552, 16, 1755, 246, 704, 332, 577, 342, 141, 304, 673, 996, 1578, 1815, 1537, 1752, 6, 874, 1043, 119, 458, 112, 877, 1121, 1045, 324]
#face 3
lista_props = [852, 1322, 348, 48, 1100, 418, 1148, 1425, 305, 1323, 62, 522, 1216, 1447, 1294, 244, 1065, 1461, 1446, 1615, 484, 1442, 558, 992, 1413, 182, 1199, 1539, 1423, 374, 400, 988, 176, 1049, 408, 1367, 559, 231, 14, 1807, 1009, 369, 1412, 1735, 419, 1092, 271, 534, 507, 290, 572, 1484, 32, 661, 986, 392, 183, 1485, 1151, 391, 1013, 1695, 705, 1483, 1830, 1428, 21, 556, 971, 1097, 628, 439, 288, 110, 1758, 1520, 376, 1241, 1240, 1198, 1282, 903, 195, 1605, 142, 1088, 586, 858, 989, 224, 265, 853, 1522, 115, 1005, 395, 43, 1052, 1474, 1269, 646, 526, 276, 1619, 898, 503, 414, 761, 766, 505, 1025, 523, 859, 1306, 627, 1593, 1430, 1104, 613, 1720, 485, 4, 36, 794, 298, 1731, 1432, 1816, 1656, 189, 173, 990, 947, 1264, 360, 302, 211, 155, 730, 1091, 715, 187, 902, 313, 631, 287, 76, 734, 1697, 1277, 292, 171, 883, 500, 910, 1136, 15, 537, 520, 1260, 1042, 1174, 1276, 442, 1719, 310, 225, 699, 1459, 286, 1001, 1587, 80, 1410, 900, 398, 931, 1491, 502, 1431, 8, 550, 969, 1706, 640, 53, 345, 1500, 371, 1302, 916, 1083, 1441, 867, 174, 1355, 579, 1702, 1503, 1756, 104, 846, 1035, 263, 1576, 619, 871, 1084, 1561, 496, 862, 1833, 588, 840, 1586, 1726, 1626, 1309, 612, 1526]
#lista_props = [1831]

In [ ]:
numeros = [1526, 404, 1594, 525, 1414, 828, 634, 1588, 1800, 706, 396, 72, 1722, 1771, 1469, 1130, 1653, 1184, 796, 1498, 303, 698, 1802, 1018, 548, 995, 541, 976, 1050, 787, 1000, 1523, 1607, 349, 307, 355, 1281, 1165, 1723, 1793, 746, 137, 882, 1356, 1559, 1470, 827, 1280, 1087, 1275, 915, 236, 943, 888, 693, 782, 1243, 783, 386, 722, 1510, 241, 450, 1513, 1612, 1456, 384, 421, 758, 941, 677, 1086, 441, 533, 311, 133, 1811, 40, 755, 576, 1694, 524, 806, 849, 880, 223, 759, 999, 788, 377, 329, 1352, 951, 571, 1597, 549, 749, 468, 1102, 1384, 611, 1290, 1580, 296, 145, 1278, 802, 985, 486, 1401, 1554, 636, 1208, 52, 212, 739, 905, 1120, 1426, 1516, 1832, 1574, 1195, 1739, 633, 832, 411, 1403, 278, 20, 1402, 1345, 424, 820, 1640, 1118, 267, 3, 1511, 539, 1307, 1108, 683, 1614, 1409, 887, 501, 901, 314, 848, 904, 417, 1225, 139, 1203, 318, 772, 407, 1393, 993, 923, 1337, 1022, 1204, 732, 459, 798, 1582, 1545, 630, 1844, 1251, 1570, 714, 545, 1077, 1099, 1438, 39, 1742, 780, 1279, 538, 919, 516, 1392, 1637, 865, 230, 617, 1633, 509, 1119, 1381, 472, 831, 733, 291, 1376, 1737, 1170, 125, 663, 589, 1115, 358, 499, 1634, 440, 773, 602, 653, 939, 1502, 1303, 1798, 1780]

In [33]:
len(lista_props)

220

In [30]:
df_intersects = None

In [31]:
# Cargar el archivo de Excel existente
wb = openpyxl.load_workbook('DATA_EST.xlsx')
# Seleccionar la hoja de trabajo
ws = wb['data']

contador = 0
for cod_prop in lista_props:
    print('inicio:', cod_prop)
    propiedad = ee.FeatureCollection(ruta_catastro)\
                .filter(ee.Filter.eq(CODIGO_PROPIEDAD, cod_prop))
    lotes_canha = propiedad.filter(ee.Filter.neq(VARIEDAD, ESTADO))
    lotes_renov = propiedad.filter(ee.Filter.eq(VARIEDAD, ESTADO))
    NDVI = ee.Image(ruta_ndvi)
    NDVI_clip = NDVI.clip(lotes_canha.geometry())
    # crea un sample de los valore de pixel del NDVI
    NDVI_values = NDVI_clip.sampleRegions(lotes_canha.geometry())
    # reduce el resultado a valores de NDVI
    pixel_values = NDVI_values.reduceColumns(ee.Reducer.toList(),['b1']).get('list').getInfo()
    # aplica metodo Jenks
    breaks = jenkspy.jenks_breaks(pixel_values, n_classes=8)
    NDVI_class = ee.Image(-1).where(NDVI.lt(breaks[1]),1)\
                        .where(NDVI.gte(breaks[1]),2)\
                        .where(NDVI.gte(breaks[2]),3)\
                        .where(NDVI.gte(breaks[3]),4)\
                        .where(NDVI.gte(breaks[4]),5)\
                        .where(NDVI.gte(breaks[5]),6)\
                        .where(NDVI.gte(breaks[6]),7)\
                        .where(NDVI.gte(breaks[7]),8)
    NDVI_class_clip = NDVI_class.clip(lotes_canha.geometry())
    clasify = NDVI_class_clip.reproject(crs="EPSG:32720", scale=10)
    vector = clasify.reduceToVectors(**{
        'geometry': lotes_canha.geometry(),
        'crs': clasify.projection(),
        'scale': 10,
        'geometryType': 'polygon',
        'eightConnected': False
    })
    
    DF_VECTOR = vector
    
    lotes_local = geemap.ee_to_geopandas(lotes_canha)
    
    lista = vector.toList(vector.size()).getInfo()
    lista_vector=[]
    for item in lista:
        dic = {'geometry':Polygon(item['geometry']['coordinates'][0]), 'count':item['properties']['count'], 'label':item['properties']['label']}
        lista_vector.append(dic)
    
    vector_local = gpd.GeoDataFrame(lista_vector)
    
    intersect = overlay(lotes_local, vector_local, how="intersection")
    intersect.crs = "EPSG:4326"
    intersect = intersect.to_crs(epsg=32720)
    intersect['area_2'] = intersect['geometry'].area/10000
    
    area_01 = intersect['area_2'].sum()
    area_02 = lotes_local[SUPERFICIE].sum()
    area_diff = (area_02 - area_01)/len(intersect)
    intersect['area_2'] = intersect['area_2'] + area_diff
    
    DF = intersect.copy()
    dina = pd.pivot_table(DF, values='area_2', index=[CODIGO_PROPIEDAD, NOM_PROPIEDAD, CODIGO_CANHERO, NOM_CANHERO, 'label'], aggfunc=np.sum)
    dina2 = dina.reset_index()
    
    dina2['tch'] = dina2['label'].apply(asig_cat)
    for i in range(0, len(dina2)):
        new_row = [dina2.iloc[i,0], dina2.iloc[i,1], dina2.iloc[i,2], dina2.iloc[i,3], dina2.iloc[i,4], dina2.iloc[i,5], dina2.iloc[i,6]]
        ws.append(new_row)
    
    df_intersects = pd.concat([df_intersects, DF], ignore_index=True)
    
    contador = contador + 1 
    print('fin: ', cod_prop)
    print('___________________CONTADOR: ', contador)
wb.save('DATA_EST.xlsx')
print('FIN......!')

inicio: 852
fin:  852
___________________CONTADOR:  1
inicio: 1322
fin:  1322
___________________CONTADOR:  2
inicio: 348
fin:  348
___________________CONTADOR:  3
inicio: 48
fin:  48
___________________CONTADOR:  4
inicio: 1100
fin:  1100
___________________CONTADOR:  5
inicio: 418
fin:  418
___________________CONTADOR:  6
inicio: 1148
fin:  1148
___________________CONTADOR:  7
inicio: 1425
fin:  1425
___________________CONTADOR:  8
inicio: 305
fin:  305
___________________CONTADOR:  9
inicio: 1323
fin:  1323
___________________CONTADOR:  10
inicio: 62
fin:  62
___________________CONTADOR:  11
inicio: 522
fin:  522
___________________CONTADOR:  12
inicio: 1216
fin:  1216
___________________CONTADOR:  13
inicio: 1447
fin:  1447
___________________CONTADOR:  14
inicio: 1294
fin:  1294
___________________CONTADOR:  15
inicio: 244
fin:  244
___________________CONTADOR:  16
inicio: 1065
fin:  1065
___________________CONTADOR:  17
inicio: 1461
fin:  1461
___________________CONTADOR:  18
ini

In [17]:
df_intersects.head()

,area,cultivo,financia,fs,id,soca,textura,unidad_01,unidad_02,unidad_03,unidad_04,unidad_05,variedad,zona,count,label,geometry,area_2
0,8.11,canha,PROPIO,16/4/2021,00007520000L3.2,1,FA,752,GUARACACHICITO--VELASQUEZ,14826,VELASQUEZ_CLAROS_JACINTO,L3.2,UCG_90-20,152,5,7,"POLYGON ((482296.408 8104190.000, 482295.764 8...",0.008011
1,14.28,canha,PROPIO,15/4/2020,00007520000L3.1,2,FA,752,GUARACACHICITO--VELASQUEZ,14826,VELASQUEZ_CLAROS_JACINTO,L3.1,UCG_90-20,152,5,7,"POLYGON ((482295.764 8104220.000, 482296.408 8...",0.034528
2,8.11,canha,PROPIO,16/4/2021,00007520000L3.2,1,FA,752,GUARACACHICITO--VELASQUEZ,14826,VELASQUEZ_CLAROS_JACINTO,L3.2,UCG_90-20,152,24,5,"POLYGON ((482300.000 8104022.518, 482299.840 8...",-0.003671
3,14.28,canha,PROPIO,15/4/2020,00007520000L3.1,2,FA,752,GUARACACHICITO--VELASQUEZ,14826,VELASQUEZ_CLAROS_JACINTO,L3.1,UCG_90-20,152,24,5,"POLYGON ((482255.560 8103940.000, 482220.000 8...",0.222638
4,8.11,canha,PROPIO,16/4/2021,00007520000L3.2,1,FA,752,GUARACACHICITO--VELASQUEZ,14826,VELASQUEZ_CLAROS_JACINTO,L3.2,UCG_90-20,152,2,6,"POLYGON ((482299.840 8104030.000, 482299.411 8...",-0.002981


In [32]:
df_intersects.to_file("INTERSECT_03.shp", driver="ESRI Shapefile")

In [91]:
Map = geemap.Map(basemap="SATELLITE")

In [92]:
Map.centerObject(propiedad.geometry(), 14)
Map.addLayer(lotes_canha.style(**vis_params_caña), {}, 'Lotes Caña')
Map.addLayer(lotes_renov.style(**vis_params_renov), {}, 'Lotes Renovación')

In [93]:
Map

Map(center=[-17.227364680451014, -62.98377937612558], controls=(WidgetControl(options=['position', 'transparen…

In [124]:
categoria = ['Nulo','Muy Bajo','Bajo Medio','Bajo','Medio','Medio Alto','Alto','Muy Alto']
colores = ['','','','','','','','']